In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: iitj_workspace1
Azure region: eastus2
Subscription id: a6dd3c75-3739-4404-a6fd-d49a8c3b1da2
Resource group: iitj_resource1


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacity-project-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_DS12_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment
from azureml.core.script_run_config import ScriptRunConfig
import os

### YOUR CODE HERE ###
# Specify parameter sampler
ps = RandomParameterSampling({
    "--C" : choice(0.0, 0.1, 2),
    "--max_iter" : choice(20, 40, 60, 100, 150, 200)
})

### YOUR CODE HERE ###
# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, delay_evaluation=5, evaluation_interval=1)

#if "training" not in os.listdir():
#    os.mkdir("./training")

# Setup environment for your training run
env = Environment.get(workspace=ws, name='AzureML-Tutorial')

### YOUR CODE HERE ###
# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory = os.getcwd(),
                      compute_target = compute_target,
                      script = 'train.py',
                      environment=env
)

### YOUR CODE HERE ###
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps,
                                     primary_metric_name = 'Accuracy',
                                     primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     run_config=src,
                                     max_total_runs=20,
                                     max_concurrent_runs=5)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

KeyError: 'log_files'

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run_hyperdrive = hyperdrive_run.get_best_run_by_primary_metric()

print("Best Run ID :", best_run_hyperdrive.id)
print("Best Run Metrics :", best_run_hyperdrive.get_metrics())

model = best_run_hyperdrive.register_model(model_name='best_run_hyperdrive_project1', 
model_path='outputs_project1/best_run_hyperdrive_model.joblib', 
properties={"Accuracy" : best_run_hyperdrive.get_metrics()["Accuracy"]})

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [4]:
from train import clean_data
from azureml.core.dataset import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Default Datastore
datastore = ws.get_default_datastore()

### YOUR DATA OBJECT HERE ###
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
df = pd.concat([x, y], axis=1)\

# Split data into train and test sets.
train, test = train_test_split(df, test_size=0.25, stratify=df.y, random_state=100)

#Register Dataset in Workspace
train_dataset = Dataset.Tabular.register_pandas_dataframe(
          dataframe=df, 
          name='bankmarketing_train', 
          description='Bank marketing dataset',
          target=datastore
          )

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=train_dataset,
    label_column_name=y,
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)

In [ ]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

### YOUR CODE HERE ###
# Retrieve and save your best automl model.
best_automl_run, best_automl_model = automl_run.get_output(return_onnx_model=True)
print(best_automl_run)

OnnxConverter.save_onnx_model(best_automl_model, 'outputs_project1/best_run_automl_model.onnx')

In [5]:
# cluster cleanup
compute_target.delete()